evaluated_model
```json
[
    {
        "question": "",
        "answer": ""
    }
]
```

In [1]:
from myutils import io
data = io.jsonload("data/eval.json")
evaluated_path = "evaluated_models/llama2-7bchat.json"
evaluated_model = io.jsonload(evaluated_path)

In [2]:
from src.double_loop import double_loop_match, double_loop_score
from tqdm import tqdm
assert len(data)==len(evaluated_model)
for data_unit, evaluated_model_unit in zip(data, evaluated_model):
    assert data_unit['question'] == evaluated_model_unit['question']
result = {
    "path": evaluated_path,
    "average_score": 0,
    "logs": []
}
sum_score = 0
for data_unit, evaluated_model_unit in tqdm(zip(data, evaluated_model)):
    question = data_unit['question']
    evaluated_answer = evaluated_model_unit['answer']
    scored_answers = [unit['answer'] for unit in data_unit['answers']]
    ys = [unit['score'] for unit in data_unit['answers']]
    xc, xs = double_loop_match(question, evaluated_answer, scored_answers)
    yc = double_loop_score(xc, xs, ys)
    sum_score += yc
    result['logs'].append({
        "xc": xc,
        "xs": xs,
        "yc": yc
    })
result['average_score'] = sum_score / len(data)
io.jsondump(result, evaluated_path+".log2")

122it [03:55,  1.93s/it]


In [ ]:
import pandas as pd
from myutils import io
import os
columns = ['model', 'language', 'memory', 'security', 'average']
for file in os.listdir("evaluated_models"):
    if file.endswith(".log2"):
        filepath = os.path.join("evaluated_models", file)
        data = io.jsonload(filepath)
        log = {
            'model': file.replace('.log2', ''),
            'language': sum([unit['yc'] for unit in data['logs']]),
            'memory': sum([unit['yc'] for unit in data['logs']]),
            'security': sum([unit['yc'] for unit in data['logs']]),
            'average': data['average_score']
        }